# Standard Implementations

Here we can see two very similar implementations of:

y = a1 + (a2&ast;x) + (a3&ast;x^2)

The first uses an array of weights and the second takes each weight as an individual input and the third is a fixed value implementation that only works with one set of weights

This results in vastly difference performance because when we access an array we need to get the value from the index which is itself arithmetic operations.

In [26]:
# Array Implementation
function SimplePoly(x, weights)
    weights[1] + (weights[2] * x) + (weights[3]* x*x)
end

# Scalar Implementation
function SimplePoly(x, a1,a2,a3)
    a1 + (a2 * x) + (a3* x*x)
end

# Hard Corded Implementation
function SimplePolyFixed(x)
    0.2 + (0.5 * x) + (1.2* x*x)
end

SimplePolyFixed (generic function with 1 method)

In [27]:
using BenchmarkTools
xInputs = rand(1000000)

function TestFixed()
    for x in xInputs
        SimplePolyFixed(x)
    end
    "Done"
end

function TestScalar()
    for x in xInputs
        SimplePoly(x, 0.2,0.5,1.2)
    end
    "Done"
end

function TestArray()
    for x in xInputs
        SimplePoly(x, [0.2,0.5,1.2])
    end
    "Done"
end

TestArray (generic function with 1 method)

In [22]:
@benchmark TestFixed()

BenchmarkTools.Trial: 
  memory estimate:  76.29 MiB
  allocs estimate:  3999490
  --------------
  minimum time:     56.712 ms (4.48% GC)
  median time:      57.769 ms (5.22% GC)
  mean time:        58.433 ms (6.16% GC)
  maximum time:     104.227 ms (39.59% GC)
  --------------
  samples:          86
  evals/sample:     1

In [29]:
@benchmark TestScalar()

BenchmarkTools.Trial: 
  memory estimate:  76.29 MiB
  allocs estimate:  3999490
  --------------
  minimum time:     61.759 ms (4.76% GC)
  median time:      64.124 ms (5.64% GC)
  mean time:        64.662 ms (6.13% GC)
  maximum time:     112.196 ms (38.66% GC)
  --------------
  samples:          78
  evals/sample:     1

In [25]:
@benchmark TestArray()

BenchmarkTools.Trial: 
  memory estimate:  183.10 MiB
  allocs estimate:  4999490
  --------------
  minimum time:     82.357 ms (9.63% GC)
  median time:      83.173 ms (10.17% GC)
  mean time:        84.161 ms (11.07% GC)
  maximum time:     128.946 ms (35.78% GC)
  --------------
  samples:          60
  evals/sample:     1

So we can see very clearly that the single inputs are much much faster without having to extract values from an array and the fixed function is even better.

(Note: Normally we would work in an array fashion and not pull back to scalar as we are here, but as a demonstration of a use-case this will do for simplicity.)

### So why is it faster?

Using @code_llvm we are able to print the generated LLVM IR from our functions (we could also use @code_native to get the assembly but LLVM IR is much more readable). As we can see the less general that we become the simpler the code gets. The array based implementation has lots of code to extract values from indices which the others simply don't need and the fixed implementation is able to read from constants rather than read in the weight values.



In [30]:
@code_llvm SimplePoly(1.5, [0.2,0.5,1.2])


; Function SimplePoly
; Location: In[26]:3
; Function Attrs: uwtable
define double @julia_SimplePoly_36381(double, %jl_value_t addrspace(10)* nonnull align 16 dereferenceable(40)) #0 {
top:
; Function getindex; {
; Location: array.jl:731
  %2 = addrspacecast %jl_value_t addrspace(10)* %1 to %jl_value_t addrspace(11)*
  %3 = bitcast %jl_value_t addrspace(11)* %2 to %jl_array_t addrspace(11)*
  %4 = getelementptr inbounds %jl_array_t, %jl_array_t addrspace(11)* %3, i64 0, i32 1
  %5 = load i64, i64 addrspace(11)* %4, align 8
  switch i64 %5, label %idxend2 [
    i64 0, label %oob
    i64 1, label %oob1
  ]

oob:                                              ; preds = %top
  %6 = alloca i64, align 8
  store i64 1, i64* %6, align 8
  %7 = addrspacecast %jl_value_t addrspace(10)* %1 to %jl_value_t addrspace(12)*
  call void @jl_bounds_error_ints(%jl_value_t addrspace(12)* %7, i64* nonnull %6, i64 1)
  unreachable

oob1:                                             ; preds = %top
  %8 = alloc

In [31]:
@code_llvm SimplePoly(1.5, 0.2,0.5,1.2)


; Function SimplePoly
; Location: In[26]:8
; Function Attrs: uwtable
define double @julia_SimplePoly_36346(double, double, double, double) #0 {
top:
; Function *; {
; Location: float.jl:399
  %4 = fmul double %0, %2
;}
; Function *; {
; Location: operators.jl:502
; Function *; {
; Location: float.jl:399
  %5 = fmul double %0, %3
  %6 = fmul double %5, %0
;}}
; Function +; {
; Location: operators.jl:502
; Function +; {
; Location: float.jl:395
  %7 = fadd double %4, %1
  %8 = fadd double %7, %6
;}}
  ret double %8
}


In [32]:
@code_llvm SimplePolyFixed(1.5)


; Function SimplePolyFixed
; Location: In[26]:13
; Function Attrs: uwtable
define double @julia_SimplePolyFixed_36383(double) #0 {
top:
; Function *; {
; Location: float.jl:399
  %1 = fmul double %0, 5.000000e-01
;}
; Function *; {
; Location: operators.jl:502
; Function *; {
; Location: float.jl:399
  %2 = fmul double %0, 1.200000e+00
  %3 = fmul double %2, %0
;}}
; Function +; {
; Location: operators.jl:502
; Function +; {
; Location: float.jl:395
  %4 = fadd double %1, 2.000000e-01
  %5 = fadd double %4, %3
;}}
  ret double %5
}


# Specialising Functions

So now let's picture this as a problem in a real-world solution. We are given an array for the weights and want to call SimplePoly for a million values of x in different places. Submitting the array as the weights is slow, but extracting the scalar values once and passing them around is not very elegant and still wont get the performance of the fixed implementation.

We could instead generate a new version of the function with the values from the array compiled in the function like this:

In [12]:
function GetSpecialisedFunction(weights)   
    @eval function(x) 
                        $(weights[1]) + 
                        ($(weights[2]) * x) +
                        ($(weights[3]) * x*x)
          end
end

const bakedInFunc = GetSpecialisedFunction(givenWeights)

#7 (generic function with 1 method)

This gives us a new function that we can call like any other and we can see below from checking the output LLVM IR that the constants are now stored within the function and the output is nearly identical to that of the hand-written hard coded example.

(*Note: To get full performance from an internally generated function it best to declare it 'const' this will improve performance by removing checks to ensure that it is still a valid function to call when it is called*)

In [34]:
@code_llvm bakedInFunc(1.0)


; Function #7
; Location: In[12]:3
; Function Attrs: uwtable
define double @"julia_#7_36071"(double) #0 {
top:
; Function *; {
; Location: float.jl:399
  %1 = fmul double %0, 5.000000e-01
;}
; Function *; {
; Location: operators.jl:502
; Function *; {
; Location: float.jl:399
  %2 = fmul double %0, 1.200000e+00
  %3 = fmul double %2, %0
;}}
; Function +; {
; Location: operators.jl:502
; Function +; {
; Location: float.jl:395
  %4 = fadd double %1, 2.000000e-01
  %5 = fadd double %4, %3
;}}
  ret double %5
}


If we now benchmark the generated solution we can see that the performance is now the same as the hard-coded function too.

We have effectively specialised and vastly improved performance over the standard array implementation

In [33]:
function TestGenerated()
    for x in xInputs
        SimplePolyFixed(x)
    end
    "Done"
end

@benchmark TestGenerated()

BenchmarkTools.Trial: 
  memory estimate:  76.29 MiB
  allocs estimate:  3999490
  --------------
  minimum time:     56.003 ms (4.72% GC)
  median time:      58.510 ms (6.33% GC)
  mean time:        60.037 ms (7.10% GC)
  maximum time:     117.918 ms (43.95% GC)
  --------------
  samples:          84
  evals/sample:     1